In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [3]:
# Load the data
data = pd.read_csv('com.csv')
pd.set_option('display.max_columns',None)
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,Unnamed: 22,Unnamed: 23
0,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,NaN,NaN
1,19/08/2000,Leicester,Aston Villa,0,0,D,0,0,D,Mike Riley,5,5,4,3,5,4,12,12,2,3,0,0,NaN,NaN
2,19/08/2000,Tottenham,Ipswich,3,1,H,2,1,H,Alan Wiley,20,15,6,5,3,4,14,13,0,0,0,0,NaN,NaN
3,20/08/2000,Man United,Newcastle,2,0,H,1,0,H,Steve Lodge,19,9,9,6,7,1,7,13,0,1,0,0,NaN,NaN
4,21/08/2000,Arsenal,Liverpool,2,0,H,1,0,H,Graham Poll,17,7,12,4,10,11,25,20,2,4,1,2,NaN,NaN


In [4]:
# Team Names
le_teams = LabelEncoder()

# Using the same label encoder for the home and away teams.
le_teams.fit(pd.concat([data['HomeTeam'],data['AwayTeam']])) 

# Using the label encoder to encode the home and away teams
data['HomeTeam_Encoded'] = le_teams.transform(data['HomeTeam'])
data['AwayTeam_Encoded'] = le_teams.transform(data['AwayTeam'])

# FTR and HTR
le_results = LabelEncoder()
le_results.fit(pd.concat([data['FTR'],data['HTR']]))

# Using the label encoder to encode the FTR and HTR
data['FTR_Encoded'] = le_results.transform(data['FTR'])
data['HTR_Encoded'] = le_results.transform(data['HTR'])

# Referee
le_referee = LabelEncoder()
le_referee.fit(data['Referee'])

data['Referee_Encoded'] = le_referee.transform(data['Referee'])
# Team Names
le_teams = LabelEncoder()

data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,Unnamed: 22,Unnamed: 23,HomeTeam_Encoded,AwayTeam_Encoded,FTR_Encoded,HTR_Encoded,Referee_Encoded
0,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,NaN,NaN,5,18,2,2,53
1,19/08/2000,Leicester,Aston Villa,0,0,D,0,0,D,Mike Riley,5,5,4,3,5,4,12,12,2,3,0,0,NaN,NaN,10,1,1,1,91
2,19/08/2000,Tottenham,Ipswich,3,1,H,2,1,H,Alan Wiley,20,15,6,5,3,4,14,13,0,0,0,0,NaN,NaN,17,9,2,2,9
3,20/08/2000,Man United,Newcastle,2,0,H,1,0,H,Steve Lodge,19,9,9,6,7,1,7,13,0,1,0,0,NaN,NaN,13,14,2,2,137
4,21/08/2000,Arsenal,Liverpool,2,0,H,1,0,H,Graham Poll,17,7,12,4,10,11,25,20,2,4,1,2,NaN,NaN,0,11,2,2,54


In [5]:
# Number of matches to look back
n = 5

# Precompute rolling averages for all attributes
def compute_pairwise_rolling(data, attributes, n):
    for attribute in attributes:
        # Group by HomeTeam and AwayTeam
        grouped = data.groupby(['HomeTeam', 'AwayTeam'])[attribute]
        # Calculate rolling averages and align the index
        rolling_avg = grouped.apply(lambda x: x.shift(1).rolling(window=n, min_periods=1).mean())
        # Align the new column with the original DataFrame's index
        data[f"n{attribute}_Pairwise"] = rolling_avg.values
    return data

# List of attributes to calculate rolling averages for
attributes = ['FTHG', 'HTHG', 'HS', 'HC', 'HF', 'HY', 'HR',
              'FTAG', 'HTAG', 'AS', 'AC', 'AF', 'AY', 'AR']

# Apply the optimized function
data = compute_pairwise_rolling(data, attributes, n)

display(data)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,Unnamed: 22,Unnamed: 23,HomeTeam_Encoded,AwayTeam_Encoded,FTR_Encoded,HTR_Encoded,Referee_Encoded,nFTHG_Pairwise,nHTHG_Pairwise,nHS_Pairwise,nHC_Pairwise,nHF_Pairwise,nHY_Pairwise,nHR_Pairwise,nFTAG_Pairwise,nHTAG_Pairwise,nAS_Pairwise,nAC_Pairwise,nAF_Pairwise,nAY_Pairwise,nAR_Pairwise
0,19/08/2000,Chelsea,West Ham,4,2,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,NaN,NaN,5,18,2,2,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19/08/2000,Leicester,Aston Villa,0,0,D,0,0,D,Mike Riley,5,5,4,3,5,4,12,12,2,3,0,0,NaN,NaN,10,1,1,1,91,1.000000,0.000000,16.000000,7.000000,12.000000,1.000000,0.0,0.00,0.000000,8.000000,4.000000,20.000000,4.000000,1.000000
2,19/08/2000,Tottenham,Ipswich,3,1,H,2,1,H,Alan Wiley,20,15,6,5,3,4,14,13,0,0,0,0,NaN,NaN,17,9,2,2,9,2.000000,0.000000,15.500000,8.000000,15.000000,2.500000,0.0,1.00,1.000000,8.000000,3.500000,17.000000,2.000000,0.500000
3,20/08/2000,Man United,Newcastle,2,0,H,1,0,H,Steve Lodge,19,9,9,6,7,1,7,13,0,1,0,0,NaN,NaN,13,14,2,2,137,2.333333,0.333333,14.666667,7.333333,13.666667,2.333333,0.0,1.00,0.666667,9.333333,5.333333,18.000000,2.333333,0.333333
4,21/08/2000,Arsenal,Liverpool,2,0,H,1,0,H,Graham Poll,17,7,12,4,10,11,25,20,2,4,1,2,NaN,NaN,0,11,2,2,54,2.250000,0.250000,14.500000,6.000000,14.250000,2.500000,0.0,0.75,0.500000,8.500000,5.500000,17.500000,2.750000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,09/11/2024,Liverpool,Aston Villa,2,0,H,1,0,H,D Coote,14,12,5,2,2,9,11,15,0,3,0,0,NaN,NaN,11,1,2,2,25,1.333333,0.333333,16.000000,7.000000,9.000000,1.333333,0.0,1.00,0.333333,9.666667,3.333333,7.666667,0.666667,0.000000
4215,10/11/2024,Man United,Leicester,3,0,H,2,0,H,P Bankes,13,6,3,5,1,5,9,5,0,1,0,0,NaN,NaN,13,10,2,2,98,1.500000,0.500000,15.250000,6.500000,8.750000,1.250000,0.0,0.75,0.250000,8.750000,3.500000,8.000000,1.000000,0.000000
4216,10/11/2024,Nott'm Forest,Newcastle,1,3,A,1,0,H,A Taylor,9,17,3,6,4,5,13,6,1,1,0,0,NaN,NaN,15,14,0,2,3,1.600000,0.600000,16.200000,6.000000,8.800000,1.200000,0.0,1.20,0.800000,8.800000,3.200000,8.200000,1.600000,0.000000
4217,10/11/2024,Tottenham,Ipswich,1,2,A,0,2,A,D England,17,8,5,3,12,2,10,19,1,5,0,0,NaN,NaN,17,9,0,0,27,1.800000,0.600000,15.400000,5.800000,8.200000,1.400000,0.0,0.80,0.600000,7.200000,2.600000,9.000000,1.800000,0.000000


In [6]:
input_features = ['HomeTeam_Encoded','AwayTeam_Encoded']
data_input = data[input_features]
display(data_input)

,HomeTeam_Encoded,AwayTeam_Encoded
0,5,18
1,10,1
2,17,9
3,13,14
4,0,11
...,...,...
4214,11,1
4215,13,10
4216,15,14
4217,17,9


In [7]:
output_features = ['FTR_Encoded']
data_output = data['FTR_Encoded']
display(data_output)

0       2
1       1
2       2
3       2
4       2
       ..
4214    2
4215    2
4216    0
4217    0
4218    1
Name: FTR_Encoded, Length: 4219, dtype: int32

In [8]:
# Features and target
X = data[input_features]
y = data['FTR_Encoded']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

# Predictions
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))



Accuracy: 0.39267767408470927
Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.54      0.44       438
           1       0.26      0.28      0.27       328
           2       0.54      0.35      0.42       627

    accuracy                           0.39      1393
   macro avg       0.39      0.39      0.38      1393
weighted avg       0.42      0.39      0.39      1393



In [9]:
# Fit the LabelEncoder during preprocessing
le_teams = LabelEncoder()
le_teams.fit(pd.concat([data['HomeTeam'], data['AwayTeam']]))

def predict_match(home_team, away_team):
    # Encode the team names
    home_team_encoded = le_teams.transform([home_team])[0]
    away_team_encoded = le_teams.transform([away_team])[0]

    # Prepare the feature vector
    match_features = pd.DataFrame([[home_team_encoded, away_team_encoded]], 
                                   columns=['HomeTeam_Encoded', 'AwayTeam_Encoded'])
    
    # Scale the features
    match_features_scaled = scaler.transform(match_features)

    # Predict the outcome
    predicted_result = svm_model.predict(match_features_scaled)

    # Map encoded result to human-readable outcome
    result_mapping = {0: "Draw", 1: "Home Win", 2: "Away Win"}
    return result_mapping[predicted_result[0]]

# Test Prediction
home_team = 'Liverpool'
away_team = 'Bournemouth'
outcome = predict_match(home_team, away_team)
print(f"The predicted outcome for {home_team} vs {away_team} is: {outcome}")



The predicted outcome for Liverpool vs Bournemouth is: Away Win
